This code is for extraction of data from GEE

In [10]:
import ee
import os
import numpy as np
import json
from tqdm import tqdm
from datetime import datetime
from utils import *
import warnings

In [11]:
# Trigger the authentication flow
ee.Authenticate()

# Initialize library
ee.Initialize(project='ee-shivaprakashssy-psetae-ka28')


In [12]:
def prepare_dataset(rpg_file, label_names, id_field, output_dir, col_id, start_date, end_date, num_per_month,cloud_cover, addNDVI, footprint_id, speckle_filter, kernel_size):
    warnings.filterwarnings('error', category=DeprecationWarning)
    start = datetime.now()

    # prepare output directory
    prepare_output(output_dir)

    # get parcel geometries & labels
    polygons, lab_rpg = parse_rpg(rpg_file, label_names, id_field)

    # dict of global metadata to store parcel dates/labels
    dates = {k:[] for k in list(polygons.keys())}
    labels = dict([(l, {}) for l in lab_rpg.keys()])
    #for geometric feature
    geom_feats = {k: {} for k in list(polygons.keys())}
    print("dictionary created")
    # counter for ignored parcels
    ignored = 0

    # iterate parcels
    for parcel_id, geometry in tqdm(polygons.items()):
        
        # get collection
        geometry = ee.Geometry.Polygon(geometry)
        #print(geometry)
        collection = get_collection(geometry, col_id, start_date, end_date, num_per_month,cloud_cover, addNDVI, footprint_id, speckle_filter, kernel_size)

        # global normalize using 2nd & 98th percentile
        collection = collection.map(normalize)
        #print(collection)
        
        # get time series array
        collection = collection.map(lambda img: img.set('temporal', ee.Image(img).reduceRegion(reducer = ee.Reducer.toList(), geometry= geometry, scale=10).values()))
        print("collection done")
        # iterate collection and return array of TxCxN. length of time series X number of channels X number of pixels in parcel
        try:
            # query pre-selected collection & make numpy array
            print("in try block")            
            np_all_dates = np.array(collection.aggregate_array('temporal').getInfo())
            print("Info on temporal")
            #print(collection.aggregate_array('temporal').getInfo())
            #print(np_all_dates.shape[-1])
            assert np_all_dates.shape[-1] > 0 
            

            
            #perimeter, shape_ratio, bbox = geom_features(geometry)
            #geom_feats[str(parcel_id)] = {
            #"perimeter": perimeter,
            #"shape_ratio": shape_ratio,
            #"bbox": bbox
            #}

            
        except:
            print('Error in parcel --------------------> {}'.format(parcel_id))
            with open(os.path.join(output_dir, 'META', 'ignored_parcels.json'), 'a+') as file:
                file.write(json.dumps(int(parcel_id))+'\n')
            ignored += 1
            
        else:
            # create date metadata
            date_series = collection.aggregate_array('doa').getInfo()
            dates[str(parcel_id)] = date_series

            # save lABELS
            for l in labels.keys():
                labels[l][parcel_id] = int(lab_rpg[l][parcel_id])
            
            #create geom_features json file. The data has been wrapped in the [] as the code in the dataset.py needs lists not the individual numbers
            perimeter, shape_ratio, bbox = geom_features(geometry)
            geom_feats[str(parcel_id)] = [int(perimeter)]
            print(geom_feats[str(parcel_id)])
            
            # save .npy 
            np.save(os.path.join(output_dir, 'DATA', str(parcel_id)), np_all_dates)


        # save global metadata (parcel dates and labels)
        with open(os.path.join(output_dir, 'META', 'geomfeat.json'), 'w') as file:
            file.write(json.dumps(geom_feats, indent=4))

        with open(os.path.join(output_dir, 'META', 'labels.json'), 'w') as file:
            file.write(json.dumps(labels, indent=4))

        with open(os.path.join(output_dir, 'META', 'dates.json'), 'w') as file:
            file.write(json.dumps(dates, indent=4))


    # get processing time
    end = datetime.now()
    print('total ignored parcels', ignored)
    print(f"\n processing time -> {end - start}")

In [13]:
rpg_file = 'training_new_wgs84.geojson'
label_names = ["CODE_GROUP"]
id_field = 'ID_PARCEL'
output_dir = 'D:\\Semester4\\ProjectVijayapur\\psetae\\GEE-to-NPY-master\\data_extraction\\sentinel\\s1\\s1_all\\traindirectory'
col_id = 'COPERNICUS/S1_GRD'
#col_id = 'COPERNICUS/S2_SR_HARMONIZED'
footprint_id = [63]
#footprint_id =['43QEU','43QFU']
start_date = '2024-09-01'
end_date = '2025-03-31'
num_per_month = 100 # You can set this to whatever value you want
cloud_cover = 10  # You can set this to whatever value you want
addNDVI = False  # Set to True or False
speckle_filter = True  # Set to True or False
kernel_size = 3

In [ ]:

prepare_dataset(rpg_file, label_names, id_field, output_dir, col_id, start_date, end_date, num_per_month,cloud_cover, addNDVI, footprint_id, speckle_filter, kernel_size)

Reading RPG . . .
reading polygons


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2501/2501 [00:00<00:00, 482119.42it/s]


dictionary created


  0%|                                                                                                                                                    | 0/2501 [00:00<?, ?it/s]

col_id that is passed COPERNICUS/S1_GRD
After s1 and s2 :
After overlap:
After num_per_month:
collection done
in try block
Info on temporal


...

[431]
col_id that is passed COPERNICUS/S1_GRD
After s1 and s2 :
After overlap:
After num_per_month:
collection done
in try block
Info on temporal


 74%|███████████████████████████████████████████████████████████████████████████████████████████████████▍                                   | 1842/2501 [2:16:28<53:43,  4.89s/it]

[487]
col_id that is passed COPERNICUS/S1_GRD
After s1 and s2 :
After overlap:
After num_per_month:
collection done
in try block
Info on temporal


 74%|███████████████████████████████████████████████████████████████████████████████████████████████████▍                                   | 1843/2501 [2:16:32<49:22,  4.50s/it]

[467]
col_id that is passed COPERNICUS/S1_GRD
After s1 and s2 :
After overlap:
After num_per_month:
collection done
in try block
Info on temporal
